# Clean `archive.txt`
A notebook for systematic fixes to `archive.txt`. Functions should not do anything if fixes are already applied.

## Clean Urls
A lot of archive urls include session ids or other issues that make analysis challenging. Let's fix it so I never have to think about it later.

In [5]:
# load archive data
with open('../data/archive.txt') as f:
    archive = f.read()

archive = archive.split('\n\n\n')
for index, game in enumerate(archive):
    url = game.split('\n')[0]
    
    # remove session id tags
    url = url[:url.find('&sid')]
    
    # put everything back together
    archive[index] = url + game[game.find('\n'):]

with open('../data/archive.txt', 'w') as f:
    f.write('\n\n\n'.join(archive))